# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True, dtype={'year': int})
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [7]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

artist_latitude     1
artist_longitude    1
dtype: int64

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
artist_id           1 non-null object
artist_latitude     0 non-null float64
artist_location     1 non-null object
artist_longitude    0 non-null float64
artist_name         1 non-null object
duration            1 non-null float64
num_songs           1 non-null int64
song_id             1 non-null object
title               1 non-null object
year                1 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 160.0+ bytes


In [9]:
df.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [10]:
df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

['SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0,
 218.93179000000001]

In [11]:
song_data = df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data

['SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0,
 218.93179000000001]

In [12]:
song_data[3] = song_data[3].item()
song_data[4] = song_data[4].item()

In [13]:
type(song_data[3])

int

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

In [16]:
artist_data = df.loc[0, artist_columns].values.tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files('data/log_data/')

In [19]:
filepath = log_files[0]

In [20]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 18 columns):
artist           390 non-null object
auth             457 non-null object
firstName        445 non-null object
gender           445 non-null object
itemInSession    457 non-null int64
lastName         445 non-null object
length           390 non-null float64
level            457 non-null object
location         445 non-null object
method           457 non-null object
page             457 non-null object
registration     445 non-null float64
sessionId        457 non-null int64
song             390 non-null object
status           457 non-null int64
ts               457 non-null int64
userAgent        445 non-null object
userId           457 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 64.3+ KB


In [22]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

artist          67
firstName       12
gender          12
lastName        12
length          67
location        12
registration    12
song            67
userAgent       12
dtype: int64

In [23]:
df[null_columns].loc[342:362]

,artist,firstName,gender,lastName,length,location,registration,song,userAgent
342,Kid Cudi,Tegan,F,Levine,227.34322,"Portland-South Portland, ME",1.540794e+12,Up Up & Away,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
343,That Petrol Emotion,Mohammad,M,Rodriguez,265.14240,"Sacramento--Roseville--Arden-Arcade, CA",1.540512e+12,Sooner Or Later,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
344,K-Ci & JoJo,Jayden,M,Graves,331.31057,"Marinette, WI-MI",1.540664e+12,All My Life,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
345,None,None,None,None,NaN,None,NaN,None,None
346,Neil Young,Tegan,F,Levine,557.13914,"Portland-South Portland, ME",1.540794e+12,Love And Only Love (1991 Live LP Version),"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
347,None,Zachary,M,Thomas,NaN,"New York-Newark-Jersey City, NY-NJ-PA",1.540645e+12,None,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; r...
348,The Smiths,Jayden,M,Graves,159.08526,"Marinette, WI-MI",1.540664e+12,Cemetery Gates,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
349,Enrique Iglesias,Jayden,M,Graves,217.99138,"Marinette, WI-MI",1.540664e+12,DÃÂ­melo,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
350,You Me At Six,Tegan,F,Levine,366.41914,"Portland-South Portland, ME",1.540794e+12,Always Attract,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
351,Kanye West,Jayden,M,Graves,242.93832,"Marinette, WI-MI",1.540664e+12,We Don't Care,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [24]:
df.size

8226

In [25]:
df = df[df['page'] == 'NextSong']
df.size

7020

In [26]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,1542154266796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,1542154657796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,1542154825796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [27]:
test = df.copy()

In [28]:
import datetime

In [29]:
test['ts'] = test['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
test.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,2018-11-14 00:03:22.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,2018-11-14 00:06:08.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,2018-11-14 00:11:06.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,2018-11-14 00:17:37.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,2018-11-14 00:20:25.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [30]:
t = df.copy()
t.ts = t['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,2018-11-14 00:03:22.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,2018-11-14 00:06:08.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,2018-11-14 00:11:06.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,2018-11-14 00:17:37.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,2018-11-14 00:20:25.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [31]:
t['ts'].dt.weekday_name.values

array(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wedn

In [32]:
time_data = (t['ts'], t['ts'].dt.hour, t['ts'].dt.day, t['ts'].dt.week, t['ts'].dt.month, t['ts'].dt.year, t['ts'].dt.weekday_name)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [33]:
import numpy as np

In [34]:
time_dict = dict(zip(column_labels, time_data))
len(time_dict)

7

In [35]:
time_df.info()

NameError: name 'time_df' is not defined

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [36]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

NameError: name 'time_df' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [37]:
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df.loc[:, user_columns]
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 0 to 456
Data columns (total 5 columns):
userId       390 non-null object
firstName    390 non-null object
lastName     390 non-null object
gender       390 non-null object
level        390 non-null object
dtypes: object(5)
memory usage: 18.3+ KB


In [38]:
user_df = user_df.drop_duplicates(subset='userId')
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 428
Data columns (total 5 columns):
userId       28 non-null object
firstName    28 non-null object
lastName     28 non-null object
gender       28 non-null object
level        28 non-null object
dtypes: object(5)
memory usage: 1.3+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [39]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [40]:
df.ts = df['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,2018-11-14 00:03:22.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,2018-11-14 00:06:08.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
2,Eddie Palmieri,Logged In,Sara,F,74,Johnson,391.83628,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Nada De Ti,200,2018-11-14 00:11:06.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
3,The Bravery,Logged In,Sara,F,75,Johnson,168.14975,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Give In,200,2018-11-14 00:17:37.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
4,K.U.K.L,Logged In,Sara,F,76,Johnson,181.28934,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Anna,200,2018-11-14 00:20:25.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [41]:
print(songplay_table_insert)


INSERT INTO songplays (
    start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);



In [42]:
print(songplay_table_create)


CREATE TABLE IF NOT EXISTS songplays (
    songplay_id SERIAL PRIMARY KEY,
    start_time timestamp NOT NULL,
    user_id int NOT NULL,
    level varchar,
    song_id varchar,
    artist_id varchar,
    session_id int NOT NULL,
    location varchar,
    user_agent varchar
);



In [43]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [44]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.